# AI Project 1

This is a course project (AI assignment) where the goal is to merge data from three different sources:
1. Clients – stored in a database (zad_domowe__clients.db, table clients).
1. Products – stored in a CSV file (zad_domowe__products.csv).
1. Invoices/Orders – available only as screenshots (zad_domowe__invoice_1.png, zad_domowe__invoice_2.png, etc.).

Your task is to combine these into a single CSV containing client, product, and invoice details. The file should include only the necessary columns with no duplicates.

Each row should represent a purchased product with its quantity, client, and unit price.

In [1]:
!pip install --quiet instructor pydantic openai

In [2]:
from pathlib import Path
import pandas as pd
import sqlite3
import base64
from getpass import getpass
from datetime import date
import instructor
from pydantic import BaseModel
from openai import OpenAI

In [3]:
openai_key = getpass("OpenAI key: ")

OpenAI key:  ········


In [14]:
RAW_DATA_PATH = Path('./data/raw')
PROCESSED_DATA_PATH = Path('./data/processed')

## Clients' data

In [63]:
CLIENTS_PATH = RAW_DATA_PATH / 'zad_domowe__clients.db'

with sqlite3.connect(CLIENTS_PATH) as conn:
    clients_df = pd.read_sql_query('SELECT * FROM clients', conn)

clients_df

,id,name,email,phone
0,1,Carla Ramsey,frypeter@example.net,(508)998-5166x12326
1,2,Howard Mcgee,jacobpatterson@example.org,001-669-353-6118x978
2,3,Eric Moore,lejacob@example.org,923-681-4646
3,4,Laura Beard,westdominique@example.org,(291)558-0966
4,5,Joshua Santiago,sheilamedina@example.com,+1-929-417-0571x74051
...,...,...,...,...
95,96,Heather Ortega,xmartin@example.org,+1-355-627-6736x91833
96,97,Olivia Barnes,dpacheco@example.net,9317151015
97,98,Tara Cabrera,geraldjames@example.net,+1-984-996-5056
98,99,Alexandra Parker,katherineharrison@example.com,001-823-577-8901


## Products' data

In [19]:
PRODUCTS_PATH = RAW_DATA_PATH / 'zad_domowe__products.csv'
products_df = pd.read_csv(PRODUCTS_PATH, sep = ';')

products_df

,id,name,price
0,1,telewizor,85
1,2,laptop,47
2,3,smartfon,60
3,4,tablet,40
4,5,słuchawki,40
5,6,kamera,92
6,7,router,48
7,8,myszka,30
8,9,klawiatura,65
9,10,pendrive,47


## Invoices' data

In [21]:
DATA_PATH = RAW_DATA_PATH

In [68]:
class InvoiceInfoItem(BaseModel):
    description: str
    product_id: int
    quantity: int
    price: float


class InvoiceInfo(BaseModel):
    company_name: str
    customer_id: int
    customer_name: str
    invoice_number: int
    date: date
    items: list[InvoiceInfoItem]


instructor_openai_client = instructor.from_openai(OpenAI(api_key=openai_key))
orders = []
for image_path in sorted(DATA_PATH.glob("zad_domowe__invoice_*.png")):
    print(f"Processing {image_path}")
    with open(image_path, "rb") as f:
        image_data = base64.b64encode(f.read()).decode('utf-8')
    
    image_url = f"data:image/png;base64,{image_data}"
    
    invoice_info = instructor_openai_client.chat.completions.create(
        # model="gpt-4o",
        model="gpt-4o-mini",
        response_model=InvoiceInfo,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Get invoice details",
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url,
                            "detail": "high"
                        },
                    },
                ],
            },
        ],
    )

    invoice_data = invoice_info.model_dump()
    
    for item in invoice_data["items"]:
        order = {
            "company_name": invoice_data["company_name"],
            "customer_id": invoice_data["customer_id"],
            "customer_name": invoice_data["customer_name"],
            "invoice_number": invoice_data["invoice_number"],
            "date": invoice_data["date"],
            "description": item["description"],
            "product_id": item["product_id"],
            "quantity": item["quantity"],
            "price": item["price"],
        }
        orders.append(order)

orders_df = pd.DataFrame(orders)

orders_df

Processing data/raw/zad_domowe__invoice_1.png
Processing data/raw/zad_domowe__invoice_10.png
Processing data/raw/zad_domowe__invoice_2.png
Processing data/raw/zad_domowe__invoice_3.png
Processing data/raw/zad_domowe__invoice_4.png
Processing data/raw/zad_domowe__invoice_5.png
Processing data/raw/zad_domowe__invoice_6.png
Processing data/raw/zad_domowe__invoice_7.png
Processing data/raw/zad_domowe__invoice_8.png
Processing data/raw/zad_domowe__invoice_9.png


,company_name,customer_id,customer_name,invoice_number,date,description,product_id,quantity,price
0,Firma XYZ,3,Eric Moore,2,2024-05-25,router,7,3,48.0
1,Firma XYZ,3,Eric Moore,2,2024-05-25,narty,17,5,55.0
2,Firma XYZ,92,Valerie Stevens,11,2024-05-02,słuchawki (id produktu 5),5,1,40.0
3,Firma XYZ,60,Troy Alexander,3,2024-05-19,ochraniacze (id produktu 16),16,4,46.0
4,Firma XYZ,78,William Velazquez,4,2024-03-26,rolki (id produktu 14),14,3,11.0
5,Firma XYZ,33,Steven Jackson,5,2024-07-23,pendrive (id produktu 10),10,4,47.0
6,Firma XYZ,33,Steven Jackson,5,2024-07-23,laptop (id produktu 2),2,2,47.0
7,Firma XYZ,8,Ronald Callahan,6,2024-03-14,słuchawki (id produktu 5),5,4,40.0
8,Firma XYZ,8,Ronald Callahan,6,2024-03-14,ochraniacze (id produktu 16),16,2,46.0
9,Firma XYZ,4,Laura Beard,7,2024-06-24,telewizor (id produktu 1),1,2,85.0


## Łączenie danych

In [69]:
company_df = orders_df.merge(products_df, left_on="product_id", right_on="id", how="left", suffixes=(None, '_products'))

company_df = company_df.merge(clients_df, left_on="customer_id", right_on="id", how="left", suffixes=(None, '_clients'))

# drop duplicates
company_df = company_df.drop(columns=['price_products', 'id', 'id_clients', 'name_clients', 'description'])

# rename for clarity
company_df.rename(columns={
    'name': 'product_name',
    'date': 'invoice_date',
    'price': 'product_price'
})

company_df.to_csv('combined.csv', index=False)